# **IF YOU GET THROUGH ME YOU'RE THE ONE**

In [ ]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg

from pydub import AudioSegment
from pydub.silence import split_on_silence

import numpy as np
import matplotlib.pyplot as plt

import xlwt
from xlwt import Workbook

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

In [ ]:
import numpy as np
def real_cepstrum(x, n=None):
# Compute the real cepstrum of a real sequence
#    x : ndarray
#        Real sequence to compute real cepstrum of.
#    n : {None, int}, optional
#        Length of the Fourier transform.
#    Returns
#    -------
#    ceps: ndarray
#        The real cepstrum.

    spectrum = np.abs(np.fft.fft(x, n=n))**2
    ceps = np.fft.ifft(np.log(spectrum))

    return ceps, spectrum

In [ ]:
#Zero Crossing Rate

def ZCR(samples, frameSize, overlap):
    wlen = len(samples)
    step = frameSize - overlap
    frameNum = math.ceil(wlen/step)
    zcr = np.zeros((frameNum,1))
    for i in range(frameNum):
        curFrame = samples[np.arange(i*step,min(i*step+frameSize,wlen))]
        #To avoid DC bias, usually we need to perform mean substraction on each frame
        curFrame = curFrame - np.mean(curFrame) #Zero-Justified
        zcr[i] = sum(curFrame[0:-1]*curFrame[1::]<=0)
    return zcr

In [ ]:
def spectral_centroid(x, samplerate=44100):
    magnitudes = np.abs(np.fft.rfft(x)) # magnitudes of positive frequencies
    length = len(x)
    freqs = np.abs(np.fft.fftfreq(length, 1.0/samplerate)[:length//2+1]) # positive frequencies
    return np.sum(magnitudes*freqs) / np.sum(magnitudes) # return weighted mean

In [ ]:
filename = 'audio/training/J_cata_02.wav'

# Workbook is created
wb = Workbook()


#List of Words
words = ["_chata_0","_chapa_0","_chave_0","_lata_0","_lapa_0","_lava_0","_casa_0","_capa_0","_cave_0","_cata_0","_chuta_0","_chupa_0","_chuva_0","_farta_0","_farpa_0","_farda_0","_ripa_0","_rita_0","_rica_0"]

#List of Dataset versions
d = {"one": 1, "two": 2}
iterable = d.keys()

#d = {"three": 3}
#iterable = d.keys()

#List of people
p = ['A','J','N']

#History At = Success / Af = Fail
At = 0
Af = 0
Jt = 0
Jf = 0
Nt = 0
Nf = 0
Ze_Ninguem = 0

TypeTest = True #ALL DATA ANALYSIS OR FILENEMAE ONLY

fs, audio1 = read(filename)
f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')


In [ ]:
#Parameters of Identification 

Pitch = [[350, 405],[406,500]] #A/(J&N)
Zcr_Avg = [[19,40],[12,18],[5,12]] #(J/N/A)

In [ ]:
# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Pitch Analysis')

#Write Data for the table
sheet1.write(1, 0, 'Andre')
sheet1.write(2, 0, 'Joaquin')
sheet1.write(3, 0, 'Nuno')
sheet1.write(0, 1, 'TRUE')
sheet1.write(0, 2, 'FALSE')
sheet1.write(0, 3, 'Success (%)')
sheet1.write(4, 0, 'Anonimus')


h=1
w=0

if (TypeTest):
    for y in words:
        for item in iterable:
            h += 1
            if (w == 9):
                w = 0        
            for x in p:
                check = x #For Validation
                
                #Write file path
                l = str(d[item])
                
                filename = 'audio/training/'+x+y+l+'.wav' #for Training DataSet
                #filename = 'audio/test/'+x+y+l+'.wav' #for Test DataSet
                
                
                #Read file path
                fs, audio1 = read(filename)
                f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')

                #Apply butterworth Filter
                wn = 1500/(fs/2)
                b3,a3 = butter(4, wn)
                audio1_filt = lfilter(b3,a3,audio1)

                #Analyze Audio (With audio Segment)
                song = AudioSegment.from_wav(filename)

                #Divide Audio in two
                db = -30
                segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
                while len(segments) != 2:
                    db = db-1
                    segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
                    if db < -90:
                        break
                
                #Find out the number of Segments
                n = len(segments)
                #print("Size of List", n)

                samples = segments[0].get_array_of_samples()
                samples = np.array(samples)

                # Calculate Power Spectrum & Cepstrum
                ceps, spec = real_cepstrum(samples, n=None)
                N = samples.shape[0]

                # Power Specturm
                power_spec = np.abs(spec[:N//2])**2

                index = np.where(power_spec == np.max(power_spec))

                abs_ceps = np.abs(ceps[:N//2])**2
                m_ceps = np.max(abs_ceps)
                
                # Calculate Zero Crossing Rate
                frameSize = 256
                overlap = 0
                zcr = ZCR(samples, frameSize, overlap)

                if (Pitch[1][0] < m_ceps < Pitch[1][1]):
                    if(Zcr_Avg[0][0] < round(np.average(zcr)) < Zcr_Avg[0][1]):
                        J = True
                        if(check == 'J'):
                            Jt += 1
                        else: 
                            Jf += 1
                    elif((Zcr_Avg[1][0] < round(np.average(zcr)) < Zcr_Avg[1][1]) and J == False ):
                        if(check == 'N'):
                            Nt += 1
                        else: 
                            Nf += 1
                    else: 
                        Ze_Ninguem += 1
                elif(Pitch[0][0] < m_ceps < Pitch[0][1]):
                    if(check == 'A'):
                        At += 1
                    else:
                        Af += 1
                else:
                    Ze_Ninguem += 1
                            
                else:
                    Ze_Ninguem += 1
                    
                J = False

elif(TypeTest == False):
    fs, audio1 = read(filename)
    check(filename[15])
    f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')
    
    # ==== UNCOMPLETE ==== STD ALONE VERSION ==== #
    

#Andre
Total = At+Af
sheet1.write(1, 1, At) 
sheet1.write(1, 2, Af)
sheet1.write(1, 3, (At/Total)*100)
                           
#Joaquin
Total = Jt+Jf
sheet1.write(2, 1, Jt) 
sheet1.write(2, 2, Jf)
sheet1.write(2, 3, (Jt/Total)*100)                           

#Nuno
Total = Nt+Nf
sheet1.write(3, 1, Nt) 
sheet1.write(3, 2, Nf)
sheet1.write(3, 3, (Nt/Total)*100)       
    
#Anonimus
sheet1.write(4, 1, Ze_Ninguem)    
wb.save('audio/TestResults.xls')

